# Notebook to clean and work with data from ADI 4277

In [1]:
!pip install stanza

     |████████████████████████████████| 574 kB 5.8 MB/s 
     |████████████████████████████████| 4.0 MB 37.3 MB/s 
     |████████████████████████████████| 175 kB 40.7 MB/s 
     |████████████████████████████████| 77 kB 5.6 MB/s 
     |████████████████████████████████| 895 kB 34.9 MB/s 
     |████████████████████████████████| 6.6 MB 32.0 MB/s 
     |████████████████████████████████| 596 kB 36.1 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=2449da0b97938b64e8cd5d09928e899b0fe9f325eac526fe23d817057457ef49
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=c68cf10bb3a9d02b305cf89c063edf6120b7a516b825f5d6065b7d425fcd2cb1
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [3]:
#Import modules
import re, nltk, os, stanza
from nltk import RegexpTokenizer
import wget
stanza.download('pt')

2022-04-24 13:44:33 INFO: Downloading default packages for language: pt (Portuguese)...


2022-04-24 13:44:41 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
#Mount Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


First, we need do clean the data from headers and stuff. Let's write a function for that (adapted from Diego Feijo: https://github.com/diego-feijo?tab=repositories)

In [4]:
def remove_headers(text):
    '''Function to clean "acórdão" from as much textual noise as possible
      Input: string
      Output: string'''
    presentation_pattern = re.compile(r'Supremo Tribunal Federal\s+Coordenadoria de Análise de Jurisprudência\s+Dje nº \d+ Divulgação \d+\/\d+\/\d+\s+Publicação \d+\/\d+\/\d+\sEmentário nº \d+-\d\s+\d+\/\d+\/\d+\s+')
    name_pattern = re.compile(r'AÇÃO DIRETA DE INCONSTITUCIONALIDADE \d\.\d+ DISTRITO FEDERAL')
    ref_pattern = re.compile(r'\nADI 4\.277\s')
    req_pattern = re.compile(r'REQTE.+')
    int_pattern = re.compile(r'INTDO.+')
    adv_pattern = re.compile(r'ADV.+')
    date_pattern =re.compile(r'\n\d+\/\d+\/\d+\s')
    header_pattern1 = re.compile(r'(^documento pode ser acessado .*?^Inteiro Teor .*?$|^\d*\nDocumento assinado digitalmente conforme.*?$|(^[A-Z]+ \d+ [A-Z]+)* / (AC|AM|AP|RS|SC|PR|RJ|SP|ES|MG|BA|SE|AL|PE|PI|CE|RN|PA|MA|RO|RR|MA|PB|TO|MS|MT|GO|DF)$)', flags=re.UNICODE | re.DOTALL | re.MULTILINE)
    header_pattern2 = re.compile(r'(^documento pode ser acessado .*?$|^Documento assinado digitalmente conforme.*?$)', flags=re.UNICODE | re.DOTALL | re.MULTILINE)
    case_number = re.compile(r'(\n\n[A-Z]+ \d+ [A-Z]+)* / (AC|AM|AP|RS|SC|PR|RJ|SP|ES|MG|BA|SE|AL|PE|PI|CE|RN|PA|MA|RO|RR|MA|PB|TO|MS|MT|GO|DF)\n\n', flags=re.UNICODE | re.DOTALL | re.MULTILINE)
    page_number = re.compile(r'(\n)?\d{3}\n', flags=re.UNICODE | re.DOTALL | re.MULTILINE)
    text = re.sub(presentation_pattern, '', text)
    text = re.sub(name_pattern, '', text)
    text = re.sub(ref_pattern, '', text)
    text = re.sub(req_pattern, '', text)
    text = re.sub(int_pattern, '', text)
    text = re.sub(adv_pattern, '', text)
    text = re.sub(date_pattern, '', text)
    text = re.sub(header_pattern1, '', text)
    text = re.sub(header_pattern2, '', text)
    text = re.sub(case_number, '', text)
    text = re.sub(page_number, '', text)
    return text

Now we need to load the data

In [5]:
#FILEPATH = 'drive/MyDrive/coding/python/local_repo/adi_4277/'
#FILENAME = 'adi_4277.txt'
DATA = wget.download('https://raw.githubusercontent.com/joaoppadua/adi_4277/main/adi_4277.txt')
with open(DATA, 'r', errors='ignore') as f:
    data_raw = f.read()

In [6]:
data_sans_headers = remove_headers(data_raw)

In [7]:
#data_sans_headers
len(data_sans_headers.split())

76357

Now, clean and tokenize the texts. First, we'll write a function. Then, we will apply it to the pre-cleaned data (with most of headers, page markers and other textual noise extracted).

In [8]:
def clean_text(text):
  ''' Cleans a pre-cleaned string, normalize and tokenize it'''
  tokenized = RegexpTokenizer(r'\w+').tokenize(text)
  tokens = [token.lower() for token in tokenized if token.isalpha()]
  return tokens

In [9]:
tokens = clean_text(data_sans_headers)

In [10]:
#Check to see if it worked
len(tokens), tokens[:50]

(74957,
 ['plenário',
  'relator',
  'min',
  'ayres',
  'britto',
  'procuradora',
  'geral',
  'da',
  'república',
  'presidente',
  'da',
  'república',
  'congresso',
  'nacional',
  'conectas',
  'direitos',
  'humanos',
  'associação',
  'brasileira',
  'de',
  'gays',
  'lésbicas',
  'e',
  'transgêneros',
  'abglt',
  'marcela',
  'cristina',
  'fogaça',
  'vieira',
  'e',
  'outro',
  'a',
  's',
  'associação',
  'de',
  'incentivo',
  'à',
  'educação',
  'e',
  'saúde',
  'de',
  'são',
  'paulo',
  'fernando',
  'quaresma',
  'de',
  'azevedo',
  'e',
  'outro',
  'a'])

Everything seems fine. Now, let's stem the data and build a generator and a Stanza pipeline

In [11]:
nlp = stanza.Pipeline('pt')
nlp_text = nlp(data_sans_headers)

2022-04-24 13:45:02 INFO: Loading these models for language: pt (Portuguese):
| Processor    | Package |
--------------------------
| tokenize     | bosque  |
| mwt          | bosque  |
| pos          | bosque  |
| lemma        | bosque  |
| depparse     | bosque  |
| constituency | cintil  |

2022-04-24 13:45:02 INFO: Use device: gpu
2022-04-24 13:45:02 INFO: Loading: tokenize
2022-04-24 13:45:19 INFO: Loading: mwt
2022-04-24 13:45:19 INFO: Loading: pos
2022-04-24 13:45:19 INFO: Loading: lemma
2022-04-24 13:45:19 INFO: Loading: depparse
2022-04-24 13:45:20 INFO: Loading: constituency
2022-04-24 13:45:20 INFO: Done loading processors!


In [12]:
lemmas = [word.lemma for sent in nlp_text.sentences for word in sent.words]
freqdist = nltk.FreqDist(lemmas)
freqdist['literal']

11

In [13]:
bigrams = nltk.bigrams(lemmas)
#next(bigrams)

In [14]:
literal_bigram = [(w1, w2) for w1, w2 in bigrams if w1 == 'literal' or w2 == 'literal']
literal_bigram

[('mesmo', 'literal'),
 ('literal', ','),
 ('seu', 'literal'),
 ('literal', 'categorização'),
 ('previsão', 'literal'),
 ('literal', '('),
 ('interpretação', 'literal'),
 ('literal', 'de'),
 ('expressão', 'literal'),
 ('literal', 'não'),
 ('expressão', 'literal'),
 ('literal', 'de'),
 ('expressão', 'literal'),
 ('literal', 'de'),
 ('expressão', 'literal'),
 ('literal', 'de'),
 ('sentido', 'literal'),
 ('literal', 'de'),
 ('expressão', 'literal'),
 ('literal', 'não'),
 ('expressão', 'literal'),
 ('literal', 'de')]

Now we will creat a nltk.Text object and print a concordance line for the word 'literal'.

In [15]:
conc_text = nltk.Text(tokens)
conc_text.concordance('literal')

Displaying 10 of 10 matches:
te como se dá já de forma até mesmo literal com ordenamentos jurídicos da comun
 do art da constituição donde a sua literal categorização com base da sociedade
eterossexuais por força da previsão literal entre homem e mulher assiste razão 
o tribunal federal df interpretação literal do texto constitucional se isso não
o em casamento adi logo a expressão literal não deixa nenhuma dúvida de que nós
es eles resultam tanto da expressão literal da lei quanto da chamada vontade do
igurar violência contra a expressão literal do texto bittencourt carlos alberto
vel dentro dos limites da expressão literal do texto rp rel min octavio gallott
o em casamento adi logo a expressão literal não deixa dúvida alguma de que nós 
como pecado nefando ou na expressão literal daqueles textos legislativos como c


As we can see, the concordance lines table does not elucidate much. It shows sentences where the idea of a "literal" intepretation or expression is a proviso on the creative interpretation that the Constitution might also allow. The prevalence of 10 in almost 75,000 tokens also does not suggest that the concept is very relevant. 

In [16]:
#Looking at another word associated with the semantic field of limits to creative intepretation
conc_text.concordance('limites')

Displaying 25 of 26 matches:
genhosa de todas busca submeter nos limites da razoabilidade e da proporcionali
os os entes da federação dentro dos limites de suas competências e nem poderia 
a república a questão transcende os limites territoriais daquela unidade federa
a disposições testamentárias e seus limites legais não podem compartilhar a pro
or desse direito que transcende aos limites intersubjetivos de um litígio entre
lhida para se viver não esbarra nos limites do direito principalmente porque o 
a expressão que se estende além dos limites da atividade política ou do espaço 
a do dispositivo em foco diante dos limites formais e materiais que a própria l
ica dos magistrados cessa diante de limites objetivos do direito posto em outra
ante quando se trata de estabelecer limites entre uma interpretação extensivain
estender a aplicação de um texto há limites a esta extensão que são atingidos t
onstituição no presente caso e seus limites em síntese o pedido das ações é par
 se indagar

Now, let's look at how prevalent is the lemma "literal" vis-à-vis the other words of the same POS

In [17]:
test_text = nlp('literal')
pos_ = [word.pos for sent in test_text.sentences for word in sent.words]
pos_

['ADJ']

In [18]:
adjectives = [word.lemma for sent in nlp_text.sentences for word in sent.words if word.pos == 'ADJ']

In [19]:
adjectives[:50]

['parcial',
 'constitucional',
 'implícito',
 'contrário',
 'jurídico',
 '3º',
 'constitucional',
 'normativo',
 'geral',
 'negativo',
 'sexual',
 'direto',
 'humano',
 'elevado',
 'normativo',
 'sexual',
 'natural',
 'empírico',
 'pétreo',
 'não-reducionista',
 'especial',
 'constitucional',
 'coloquial',
 'proverbial',
 'doméstico',
 'formal',
 'heteroafetivo',
 'homoafetivo',
 'família',
 'heteroafetivo',
 'cartorário',
 'civil',
 'religioso',
 'privado',
 'adulto',
 'civil',
 'necessário',
 'tricotômico',
 'familiar',
 'principal',
 'institucional',
 'fundamental',
 'privado',
 '5º',
 'heteroafetivo',
 'homoafetivo',
 'igual',
 'subjetivo',
 'autonomizada',
 'central']

In [20]:
num_adjs = len(adjectives)
num_adjs, len(set(adjectives))

(7013, 1240)

In [21]:
normalized_literal = freqdist['literal'] / num_adjs
normalized_literal

0.0015685156138599743

In [23]:
freqdist_adjs = nltk.FreqDist(adjectives)
most_common_adjs = freqdist_adjs.most_common()
most_common_adjs[0]

('constitucional', 322)

In [24]:
ratio_adjs = most_common_adjs[0][1] / freqdist_adjs['literal']
ratio_adjs

29.272727272727273

Compared to the most commons adjective, "constitucional", "literal" is over 29 times less likely to appear in a modifier slot. So there is limited attention to the concept conveyed by literal interpretaions or the literal interpretation as a limit to other types of constitutional sense making procedures. 